Valsecchi et al. (2014) show that, in the limit of large $M_{\rm p}/M_\star$ and conservative mass transfer, $a/a_0 = \left(M_{\rm p}/M_{\rm p,\ 0} \right)^{-1/3}$ (Eqn 14), where $a/a_0$ is the final/initial semi-major axis and $M_{\rm p}/M_{\rm p,\ 0}$ the final/initial planetary mass. Based on this equation, where would we expect the remnants of hot Jupiters that have lost their atmospheres to end up?

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from astropy.units import cds

In [27]:
def calc_smallq_roche_limit(q, Rp=0.1):
    return (q/3.)**(-1./3)*Rp

#Returns period or semi-major axis, given the other
#  P in days, a in AU, and Mstar in solar masses
def Keplers_Third_Law(P=None, a=None, Mstar=1.*cds.Msun):

    bigG = 4.*np.pi*np.pi/cds.Msun*(cds.au*cds.au*cds.au)/(cds.yr*cds.yr)

    if((P is None) and (a is None)):
        raise ValueError("P or a must be given!")
    elif(a is not None):
        return (a*a*a/(bigG*Mstar/(4.*np.pi*np.pi)))**(1./2)
    elif(P is not None):
        return (P*P*(bigG*Mstar/(4.*np.pi*np.pi)))**(1./3)

In [42]:
#typical hot Jupiter
Mp = 1.*cds.Mjup
Rp = 1.*cds.Rjup
Mcore = 10.*cds.Mgeo

#the Sun
Ms = 1.*cds.Msun
Rs = 1.*cds.Rsun

q = (Mp.to('kg')/Ms.to('kg')).value

a_Roche = calc_smallq_roche_limit(q, Rp=Rp)

#So where would the planet end up?
a = a_Roche*(Mcore/Mp)**(-1./3)
P = Keplers_Third_Law(a=a, Mstar=Ms)
print(a.to('au').value, P.to('day').value)

(0.02217335479484554, 1.2059723035426975)


In [ ]:
Mps = 10**(np.linspace(-1., 1., 100))*cds.Mjup